In [ ]:
!pip install -r https://raw.githubusercontent.com/bentoml/BentoML/main/examples/quickstart/requirements.txt

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df1 = pd.read_csv('https://raw.githubusercontent.com/arinaaandreeva/Datasets/main/CO2_emission_2008_13.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/arinaaandreeva/Datasets/main/CO2_emission_2014.csv')
df = pd.concat([df1, df2], axis=0)
print(df1.shape, df2.shape, df.shape)

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [ ]:
strings = list(df.dtypes[df.dtypes == 'object'].index)
strings = strings[1:len(strings)]
strings

In [ ]:
for i in strings:
  df[i]=df[i].apply(lambda x:x.replace('..',''))
  df[i]=df[i].apply(lambda x:x.replace(',',''))
  df[i]=df[i].apply(pd.to_numeric)

df['year']=df['year'].astype(object)
df = df.fillna(0)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

In [ ]:
ohe = OneHotEncoder()
transformed = ohe.fit_transform(df[['country', 'year']])
print(transformed.toarray())

In [ ]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [ ]:
y_train = df_train.co2_emission_metrics_per_capita.values
y_val = df_val.co2_emission_metrics_per_capita.values
y_test = df_test.co2_emission_metrics_per_capita.values

del df_train['co2_emission_metrics_per_capita']
del df_val['co2_emission_metrics_per_capita']
del df_test['co2_emission_metrics_per_capita']

In [ ]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)

X_train = df_train.reset_index(drop=True)
X_train_dict = X_train.to_dict(orient='records')
X_train = dv.fit_transform(X_train_dict)

X_val = df_val.reset_index(drop=True)
X_val_dict = X_val.to_dict(orient='records')
X_val = dv.fit_transform(X_val_dict)

X_test = df_test.reset_index(drop=True)
X_test_dict = X_test.to_dict(orient='records')
X_test = dv.fit_transform(X_test_dict)

In [ ]:
y_full_train = df_full_train['co2_emission_metrics_per_capita'].astype(int)
del df_full_train['co2_emission_metrics_per_capita']
dicts_full_train = df_full_train.to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X_full_train = dv.fit_transform(dicts_full_train)

dicts_test = df_test.to_dict(orient='records')
X_test = dv.transform(dicts_test)

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost.sklearn import XGBClassifier

In [ ]:
dfulltrain = xgb.DMatrix(X_full_train, label=y_full_train,
                    feature_names=dv.get_feature_names())

dtest = xgb.DMatrix(X_test, feature_names=dv.get_feature_names())

##  Model Training

In [ ]:

model = xgb.XGBRegressor(learning_rate = 0.5, n_estimators = 200, seed = 25)
 
model.fit(X_train, y_train)


Save the `clf` model instance to BentoML local model store:

In [ ]:
import bentoml

bentoml.sklearn.save_model("co2_emission_model", model)

Models saved can be accessed via `bentoml models` CLI command:

In [ ]:
!bentoml models get co2_emission_model:latest

In [ ]:
!bentoml models list

To verify that the saved model can be loaded correctly:

In [ ]:
loaded_model = bentoml.sklearn.load_model("co2_emission_model:latest")

loaded_model.predict(X_test)

In [ ]:
# Create a Runner instance:
model_runner = bentoml.sklearn.get("co2_emission_model:latest").to_runner()

# Runner#init_local initializes the model in current process, this is meant for development and testing only:
model_runner.init_local()

# This should yield the same result as the loaded model:
model_runner.predict.run(X_test)

## Serving the model


In [ ]:
%%writefile service.py
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray

model_runner = bentoml.sklearn.get("co2_emission_model:latest").to_runner()

svc = bentoml.Service("co2_emission_classifier", runners=[model_runner])

@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    return model_runner.predict.run(input_series)


In [ ]:
!bentoml serve service.py:svc --reload

### Build Bento for deployment

In [ ]:
%%writefile bentofile.yaml
service: "service.py:svc"
labels:
  owner: Andreeva_Arina
  project: Prediction of CO2 emission
include:
- "*.py"
python:
  packages:
    - scikit-learn
    - pandas

In [ ]:
!pip install pydantic

In [ ]:
from pydantic import BaseModel

In [ ]:
# Create pydantic base class to create data schema for validation
class CreditApplication(BaseModel):
    Country : object
    Year : object
    GDP_Growth : float
    CO2_Emission_Metrics_per_capita : float
    Electricity_access : float
    Urban_population : float
    Population_growth : float
    Urban_population_growth_annual : float
    Live_stock_production : float
    Forest_area : float
    Renewable_energy_use : float
    Energy_use_kg_of_oil_per_capita : float
    Population_total : float

In [ ]:
from bentoml.io import JSON

In [ ]:
'''model_runner = bentoml.sklearn.get("co2_emission_model:latest").to_runner()

svc = bentoml.Service("co2_emission_classifier", runners=[model_runner])

@svc.api(input=JSON(pydantic_model=CreditApplication), output=JSON()) # decorate endpoint as in json format for input and output
def classify(credit_application):
    # transform pydantic class to dict to extract key-value pairs 
    application = credit_application.dict()
    # transform data from client using dictvectorizer
    vector = dv.transform(application)
    # make predictions using 'runner.predict.run(input)' instead of 'model.predict'
    prediction = model_runner.predict.run(vector) '''

In [ ]:
!bentoml build

## Containerize and Deployment

In [ ]:
!sudo apt install apt-transport-https ca-certificates curl software-properties-common -qq

In [ ]:
!curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

In [ ]:
!sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"

In [ ]:
!sudo apt install docker-ce

In [ ]:
!docker

In [ ]:
!bentoml containerize co2_emission_model:latest

In [ ]:
!docker run -p 3000:3000 co2_emission_model:3yult2takw6cwasc